In [ ]:
# Imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from sklearn.model_selection import train_test_split

sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (10,6)

## Cargar `Folds.csv` y extraer metadatos

In [ ]:
# Ajusta la ruta si es necesario (Kaggle: ../input/...)
fold_csv = '../input/breakhis/Folds.csv'
if not os.path.exists(fold_csv):
    print('Advertencia: no se encontró', fold_csv, '— revisa la ruta.')

fold_df = pd.read_csv(fold_csv)
# Renombrar columna filename a path si es necesario
if 'filename' in fold_df.columns:
    fold_df = fold_df.rename(columns={'filename':'path'})

# Extraer filename y label como en el notebook original
fold_df['filename'] = fold_df['path'].apply(lambda x: x.split('/')[-1])
# La etiqueta original se encuentra en la posición 3 del path
fold_df['label'] = fold_df['path'].apply(lambda x: x.split('/')[3])
fold_df.head(3)

## Distribución global de clases (Folds.csv)

In [ ]:
# Conteo de labels
plt.figure(figsize=(8,5))
sns.countplot(fold_df['label'])
plt.xlabel('Clase')
plt.title('Distribución global de clases (Folds.csv)')
plt.show()

## Muestras: 40 imágenes benign y 40 imágenes malignant desde `../Cancer/`

In [ ]:
# Asegurarse de que el pool ../Cancer/ exista (creado por DataSimplification)
pool_dir = '../Cancer/'
if not os.path.exists(pool_dir):
    print('Advertencia: no existe', pool_dir, '. Ejecuta primero DataSimplification.ipynb para crear el pool.')

# Generar listas filtradas por prefijo (usado en la notebook original)
benign_files = [f for f in os.listdir(pool_dir) if f.startswith('benign_')] if os.path.exists(pool_dir) else []
malign_files = [f for f in os.listdir(pool_dir) if f.startswith('malignant_')] if os.path.exists(pool_dir) else []

# Mostrar hasta 40 ejemplos de cada clase (4x10 grid)
n_show = 40
if len(benign_files) > 0:
    plt.figure(figsize=(30,10))
    for i, fname in enumerate(benign_files[:n_show]):
        try:
            img = cv2.imread(os.path.join(pool_dir, fname))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            plt.subplot(4,10,i+1)
            plt.imshow(img)
            plt.axis('off')
        except Exception:
            pass
    plt.suptitle('Ejemplos Benign (pool ../Cancer)')
    plt.show()
else:
    print('No hay archivos benign en', pool_dir)

if len(malign_files) > 0:
    plt.figure(figsize=(30,10))
    for i, fname in enumerate(malign_files[:n_show]):
        try:
            img = cv2.imread(os.path.join(pool_dir, fname))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            plt.subplot(4,10,i+1)
            plt.imshow(img)
            plt.axis('off')
        except Exception:
            pass
    plt.suptitle('Ejemplos Malignant (pool ../Cancer)')
    plt.show()
else:
    print('No hay archivos malignant en', pool_dir)

## Construir `df` desde el pool `../Cancer/` y preparar splits (para visualizar distribuciones)

In [ ]:
# Crear DataFrame df a partir de los archivos del pool (solo lectura)
if os.path.exists(pool_dir):
    df = pd.DataFrame(os.listdir(pool_dir), columns=['file_loc'])
    df['label'] = df['file_loc'].apply(lambda x: x.split('_')[0])
    df['class'] = df['label'].apply(lambda x: 0 if x == 'benign' else 1)
    df.set_index('file_loc', inplace=True)
    df.head()
else:
    print('Pool no encontrado, omitiendo creación de df')

In [ ]:
# Realizar splits y mostrar conteos por split (si df existe)
if 'df' in globals():
    data_train_and_val, data_test = train_test_split(df, test_size=0.1, random_state=47)
    data_train, data_val = train_test_split(data_train_and_val, test_size=0.1, random_state=47)
    plt.figure(figsize=(18,5))
    plt.subplot(1,3,1)
    sns.countplot(data_train['class'])
    plt.title('Training set')
    plt.subplot(1,3,2)
    sns.countplot(data_val['class'])
    plt.title('Validation set')
    plt.subplot(1,3,3)
    sns.countplot(data_test['class'])
    plt.title('Test set')
    plt.show()
else:
    print('df no está disponible — no se puede mostrar distribuciones por split')

---
Fin del notebook EDA.

Ejecuta las celdas en orden. Si faltan imágenes en `../Cancer/` ejecuta primero `DataSimplification.ipynb`.